In [6]:
# from tensorflow.keras.appliications import EfficientNetB0
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras import layers, models
from tensorflow.keras.metrics import AUC,Precision,Recall
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
import tensorflow as tf
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomTranslation, RandomZoom
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,Tensorboard

from tensorflow.keras.mixed_precision import set_global_policy
import numpy as np
import os

gpus = tf.config.list_physical_devices('GPU')

if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu,True)
    except RuntimeError as e:
        print(e)


In [20]:

os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
set_global_policy('mixed_float16')
tf.keras.backend.clear_session()

data_augmentation = tf.keras.Sequential([
    RandomFlip("horizontal"),
    RandomTranslation(0.1, 0.1),
    RandomZoom(0.1),
    RandomRotation(0.05)
])
with tf.device('/GPU:0'):
    base_model = EfficientNetB0(include_top=False, weights='imagenet')
    base_model.trainable = True  

    
    inputs = layers.Input(shape=(224, 224, 1))
    x = data_augmentation(inputs)
    x = layers.Lambda(lambda t: tf.image.grayscale_to_rgb(t))(x)
    # x = tf.image.grayscale_to_rgb(x)           # Replace Conv2D with this
    x = preprocess_input(x)
    x = base_model(x)
    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(7, activation='softmax', dtype='float32')(x)
    model = models.Model(inputs, outputs)
    for layer in base_model.layers[:119]:
        layer.trainable = False

    for i, layer in enumerate(base_model.layers):
        if layer.trainable:
            print(i, layer.name, layer.trainable)

model.summary()

for i, layer in enumerate(model.layers):
    if layer.trainable:
        print(i, layer.name, layer.trainable)



119 block4c_add True
120 block5a_expand_conv True
121 block5a_expand_bn True
122 block5a_expand_activation True
123 block5a_dwconv True
124 block5a_bn True
125 block5a_activation True
126 block5a_se_squeeze True
127 block5a_se_reshape True
128 block5a_se_reduce True
129 block5a_se_expand True
130 block5a_se_excite True
131 block5a_project_conv True
132 block5a_project_bn True
133 block5b_expand_conv True
134 block5b_expand_bn True
135 block5b_expand_activation True
136 block5b_dwconv True
137 block5b_bn True
138 block5b_activation True
139 block5b_se_squeeze True
140 block5b_se_reshape True
141 block5b_se_reduce True
142 block5b_se_expand True
143 block5b_se_excite True
144 block5b_project_conv True
145 block5b_project_bn True
146 block5b_drop True
147 block5b_add True
148 block5c_expand_conv True
149 block5c_expand_bn True
150 block5c_expand_activation True
151 block5c_dwconv True
152 block5c_bn True
153 block5c_activation True
154 block5c_se_squeeze True
155 block5c_se_reshape True
1

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 224, 224, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda (Lambda)                 │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 7)              │         8,967 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,058,538 (15.48 MB)

 Trainable params: 3,707,855 (14.14 MB)

 Non-trainable params: 350,683 (1.34 MB)

0 input_layer_1 True
1 sequential True
2 lambda True
3 efficientnetb0 True
4 global_average_pooling2d True
5 dense True


In [10]:
X_train = np.load("../data/numpy/X_train.npy")
Y_train = np.load("../data/numpy/Y_train.npy")

X_val = np.load("../data/numpy/X_valid.npy")
Y_val = np.load("../data/numpy/Y_valid.npy")

In [21]:

model.compile(optimizer=Adam(learning_rate=1e-4), loss = "categorical_crossentropy", metrics = ["accuracy"])

In [18]:
my_callbacks = [
    ModelCheckpoint(
        monitor = "val_loss" ,# commonly uses valloss why? cuz secret
        verbose =1,
        filepath = "../checkpoint/model3_{epoch:02d}.keras",
        save_best_only = True,
        save_weights_only = False
    ),

    ReduceLROnPlateau(
        monitor = "val_loss",
        factor = 0.5,
        patience = 5,
        verbose = 1,
        min_lr = 1e-7
    )
    # EarlyStopping(
    #     monitor = 'val_loss',
    #     patience = 5,
    #     restore_best_weights =True
    # )
    
]

train_ds = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).batch(8)
val_ds = tf.data.Dataset.from_tensor_slices((X_val, Y_val)).batch(8)



In [25]:
from sklearn.utils import class_weight
# import numpy as np
y_int = np.argmax(Y_train,axis = 1)
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes = np.unique(y_int),
    y = y_int
)
class_weights = dict(enumerate(class_weights))
history = model.fit(
    train_ds,
    validation_data=val_ds,
    # initial_epoch = 31,
    epochs=25,
    # batch_size = 8,
    verbose = 1,
    callbacks = my_callbacks,
    class_weight = class_weights
)

Epoch 1/25


E0000 00:00:1746429666.112399    1168 meta_optimizer.cc:966] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/functional_1_1/efficientnetb0_1/block2b_drop_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


1574/1574 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step - accuracy: 0.2289 - loss: 1.8990
Epoch 1: val_loss did not improve from 1.58305
1574/1574 ━━━━━━━━━━━━━━━━━━━━ 239s 130ms/step - accuracy: 0.2289 - loss: 1.8989 - val_accuracy: 0.3176 - val_loss: 1.7892 - learning_rate: 1.0000e-04
Epoch 2/25
1574/1574 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - accuracy: 0.2612 - loss: 1.7460
Epoch 2: val_loss did not improve from 1.58305
1574/1574 ━━━━━━━━━━━━━━━━━━━━ 269s 135ms/step - accuracy: 0.2612 - loss: 1.7460 - val_accuracy: 0.3848 - val_loss: 1.6052 - learning_rate: 1.0000e-04
Epoch 3/25
1574/1574 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step - accuracy: 0.2835 - loss: 1.6849
Epoch 3: val_loss did not improve from 1.58305
1574/1574 ━━━━━━━━━━━━━━━━━━━━ 206s 131ms/step - accuracy: 0.2835 - loss: 1.6849 - val_accuracy: 0.2546 - val_loss: 1.8315 - learning_rate: 1.0000e-04
Epoch 4/25
1574/1574 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step - accuracy: 0.2931 - loss: 1.6630
Epoch 4: val_loss did not improve from 1.58305
1574/1574 

KeyboardInterrupt: 